In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import uuid
from faker import Faker
import json

In [44]:
import os
os.path.abspath("")

'C:\\Users\\ander\\learn_dbt'

In [2]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
fake = Faker()
Faker.seed(42)

In [3]:
# Configuration
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2024, 12, 31)

START_DATE = START_DATE.date()
END_DATE = END_DATE.date()

NUM_CUSTOMERS = 1000
NUM_SUBSCRIPTIONS = 1200  # Some customers have multiple subscriptions

In [4]:
print("Generating Tailscale Analytics Test Dataset...")
print("=" * 50)

Generating Tailscale Analytics Test Dataset...


In [5]:
# 1. CUSTOMERS TABLE
print("Generating customers data...")
customers = []
company_types = ['Startup', 'SMB', 'Mid-Market', 'Enterprise']
industries = ['Technology', 'Healthcare', 'Finance', 'Manufacturing', 'Education', 'Retail', 'Media']
acquisition_channels = ['Organic', 'Paid Search', 'Content Marketing', 'Referral', 'Direct Sales', 'Partner']

Generating customers data...


In [6]:
for i in range(NUM_CUSTOMERS):
    customer = {
        'customer_id': f"cust_{str(uuid.uuid4())[:8]}",
        'company_name': fake.company(),
        'company_size': random.choice(['1-10', '11-50', '51-200', '201-1000', '1000+']),
        'company_type': random.choice(company_types),
        'industry': random.choice(industries),
        'country': fake.country_code(),
        'created_at': fake.date_between(START_DATE, END_DATE),
        'acquisition_channel': random.choice(acquisition_channels),
        'is_active': random.choices([True, False], weights=[0.85, 0.15])[0]
    }
    customers.append(customer)

In [7]:
customers_df = pd.DataFrame(customers)

In [8]:
# 2. SUBSCRIPTION PLANS
print("Generating subscription plans...")
plans = [
    {'plan_id': 'personal', 'plan_name': 'Personal', 'price_monthly': 0, 'price_annual': 0, 'max_devices': 20},
    {'plan_id': 'premium', 'plan_name': 'Premium', 'price_monthly': 6, 'price_annual': 60, 'max_devices': 100},
    {'plan_id': 'team', 'plan_name': 'Team', 'price_monthly': 6, 'price_annual': 60, 'max_devices': 100},  # per user
    {'plan_id': 'business', 'plan_name': 'Business', 'price_monthly': 18, 'price_annual': 180, 'max_devices': 200},  # per user
    {'plan_id': 'enterprise', 'plan_name': 'Enterprise', 'price_monthly': 48, 'price_annual': 480, 'max_devices': 1000}  # per user
]

Generating subscription plans...


In [9]:
plans_df = pd.DataFrame(plans)

In [10]:
# 3. SUBSCRIPTIONS
print("Generating subscriptions data...")
subscriptions = []
billing_cycles = ['monthly', 'annual']

Generating subscriptions data...


In [11]:
for i in range(NUM_SUBSCRIPTIONS):
    customer_id = random.choice(customers_df['customer_id'].tolist())
    plan = random.choice(plans)
    billing_cycle = random.choice(billing_cycles)
    
    # Weight plan selection based on company size
    customer_size = customers_df[customers_df['customer_id'] == customer_id]['company_size'].iloc[0]
    if customer_size in ['1-10']:
        plan = random.choices(plans, weights=[0.4, 0.3, 0.2, 0.08, 0.02])[0]
    elif customer_size in ['11-50']:
        plan = random.choices(plans, weights=[0.1, 0.2, 0.4, 0.25, 0.05])[0]
    elif customer_size in ['51-200']:
        plan = random.choices(plans, weights=[0.05, 0.1, 0.3, 0.4, 0.15])[0]
    else:
        plan = random.choices(plans, weights=[0.02, 0.05, 0.2, 0.4, 0.33])[0]
    
    start_date = fake.date_between(START_DATE, END_DATE - timedelta(days=30))
    
    # Determine if subscription is still active
    is_churned = random.choices([True, False], weights=[0.2, 0.8])[0]
    if is_churned:
        end_date = fake.date_between(start_date + timedelta(days=30), END_DATE)
        status = 'churned'
    else:
        end_date = None
        status = 'active'
    
    # User count for team/business/enterprise plans
    if plan['plan_id'] in ['team', 'business', 'enterprise']:
        if customer_size == '1-10':
            user_count = random.randint(2, 8)
        elif customer_size == '11-50':
            user_count = random.randint(5, 35)
        elif customer_size == '51-200':
            user_count = random.randint(20, 150)
        elif customer_size == '201-1000':
            user_count = random.randint(50, 500)
        else:
            user_count = random.randint(100, 2000)
    else:
        user_count = 1
    
    subscription = {
        'subscription_id': f"sub_{str(uuid.uuid4())[:8]}",
        'customer_id': customer_id,
        'plan_id': plan['plan_id'],
        'billing_cycle': billing_cycle,
        'user_count': user_count,
        'start_date': start_date,
        'end_date': end_date,
        'status': status,
        'created_at': start_date,
        'updated_at': fake.date_between(start_date, END_DATE)
    }
    subscriptions.append(subscription)

In [12]:
subscriptions_df = pd.DataFrame(subscriptions)

In [13]:
subscriptions_df.dtypes

subscription_id    object
customer_id        object
plan_id            object
billing_cycle      object
user_count          int64
start_date         object
end_date           object
status             object
created_at         object
updated_at         object
dtype: object

In [14]:
subscriptions_df.head(-4)

,subscription_id,customer_id,plan_id,billing_cycle,user_count,start_date,end_date,status,created_at,updated_at
0,sub_83cefbca,cust_3f380b72,business,annual,915,2024-07-27,None,active,2024-07-27,2024-12-28
1,sub_bc8e8bfd,cust_78e35adb,enterprise,monthly,408,2024-08-11,None,active,2024-08-11,2024-09-28
2,sub_fb1a76b8,cust_0a9a3d5d,premium,monthly,1,2024-09-26,None,active,2024-09-26,2024-11-07
3,sub_0a4ddfb8,cust_408dbc4b,personal,monthly,1,2024-05-26,None,active,2024-05-26,2024-12-30
4,sub_7a4d54e9,cust_7c6b02b2,premium,monthly,1,2024-07-06,None,active,2024-07-06,2024-12-02
...,...,...,...,...,...,...,...,...,...,...
1191,sub_762b103b,cust_a2f84162,business,monthly,95,2024-07-21,None,active,2024-07-21,2024-11-19
1192,sub_71768fd8,cust_afcce1cd,business,monthly,1753,2024-01-09,2024-05-31,churned,2024-01-09,2024-06-18
1193,sub_ece68b01,cust_441e05f1,enterprise,annual,188,2024-09-22,2024-12-13,churned,2024-09-22,2024-10-17
1194,sub_2b1811b4,cust_5aa5926c,business,annual,143,2024-07-03,None,active,2024-07-03,2024-09-08


In [15]:
subscriptions_df['end_date'].value_counts()

end_date
2024-12-30    8
2024-12-09    6
2024-12-17    6
2024-12-29    5
2024-10-22    5
             ..
2024-09-08    1
2024-08-26    1
2024-10-07    1
2024-06-10    1
2024-12-12    1
Name: count, Length: 139, dtype: int64

In [16]:
# 4. INVOICES
print("Generating invoices data...")
invoices = []
invoice_statuses = ['draft', 'sent', 'paid', 'overdue', 'cancelled']

Generating invoices data...


In [17]:
type(subscription['end_date'])

datetime.date

In [18]:
for _, subscription in subscriptions_df.iterrows():
    # Generate invoices for each subscription
    current_date = subscription['start_date']
#    end_date = subscription['end_date'] if subscription['end_date'] else END_DATE
    end_date = subscription['end_date'] if subscription['end_date'] else END_DATE
    
    billing_frequency = 12 if subscription['billing_cycle'] == 'monthly' else 1
    days_between_invoices = 30 if subscription['billing_cycle'] == 'monthly' else 365

    #    while current_date <= end_date:
    while current_date <= end_date:
        plan_info = plans_df[plans_df['plan_id'] == subscription['plan_id']].iloc[0]
        
        if subscription['billing_cycle'] == 'monthly':
            amount = plan_info['price_monthly'] * subscription['user_count']
        else:
            amount = plan_info['price_annual'] * subscription['user_count']
        
        # Add some random discounts
        discount_percent = random.choices([0, 10, 20, 25], weights=[0.7, 0.15, 0.1, 0.05])[0]
        discount_amount = amount * (discount_percent / 100)
        final_amount = amount - discount_amount
        
        # Determine invoice status
        if current_date < datetime.now().date() - timedelta(days=60):
            status = random.choices(['paid', 'overdue'], weights=[0.95, 0.05])[0]
        elif current_date < datetime.now().date() - timedelta(days=30):
            status = random.choices(['paid', 'sent', 'overdue'], weights=[0.9, 0.08, 0.02])[0]
        else:
            status = random.choices(['paid', 'sent', 'draft'], weights=[0.7, 0.25, 0.05])[0]
        
        paid_date = None
        if status == 'paid':
            paid_date = current_date + timedelta(days=random.randint(1, 30))
        
        invoice = {
            'invoice_id': f"inv_{str(uuid.uuid4())[:8]}",
            'subscription_id': subscription['subscription_id'],
            'customer_id': subscription['customer_id'],
            'invoice_date': current_date,
            'due_date': current_date + timedelta(days=30),
            'paid_date': paid_date,
            'amount': amount,
            'discount_amount': discount_amount,
            'final_amount': final_amount,
            'status': status,
            'billing_period_start': current_date,
            'billing_period_end': current_date + timedelta(days=days_between_invoices-1),
            'created_at': current_date
        }
        invoices.append(invoice)
        
        # Move to next billing period
        current_date += timedelta(days=days_between_invoices)

In [19]:
invoices_df = pd.DataFrame(invoices)

In [20]:
# 5. PAYMENTS (Stripe-like data)
print("Generating payments data...")
payments = []
payment_methods = ['card', 'ach', 'wire_transfer']

Generating payments data...


In [21]:
for _, invoice in invoices_df[invoices_df['status'] == 'paid'].iterrows():
    payment = {
        'payment_id': f"pay_{str(uuid.uuid4())[:8]}",
        'invoice_id': invoice['invoice_id'],
        'customer_id': invoice['customer_id'],
        'amount': invoice['final_amount'],
        'currency': 'USD',
        'payment_method': random.choice(payment_methods),
        'payment_date': invoice['paid_date'],
        'stripe_charge_id': f"ch_{str(uuid.uuid4())[:16]}",
        'status': 'succeeded',
        'created_at': invoice['paid_date']
    }
    payments.append(payment)

In [22]:
payments_df = pd.DataFrame(payments)

In [23]:
# 6. USAGE DATA (for potential usage-based billing)
print("Generating usage data...")
usage_data = []

Generating usage data...


In [24]:
for _, subscription in subscriptions_df.iterrows():
    # Generate monthly usage data
    current_date = subscription['start_date'].replace(day=1)  # Start of month
    end_date = subscription['end_date'] if subscription['end_date'] else END_DATE
    
    while current_date <= end_date:
        # Simulate device connections, data transfer, etc.
        base_devices = subscription['user_count'] * random.uniform(1.2, 3.5)
        devices_connected = max(1, int(base_devices * random.uniform(0.7, 1.3)))
        
        usage = {
            'usage_id': f"usage_{str(uuid.uuid4())[:8]}",
            'subscription_id': subscription['subscription_id'],
            'customer_id': subscription['customer_id'],
            'month': current_date,
            'devices_connected': devices_connected,
            'data_transfer_gb': round(devices_connected * random.uniform(10, 500), 2),
            'active_users': min(subscription['user_count'], devices_connected),
            'connection_hours': round(devices_connected * random.uniform(100, 500), 2),
            'created_at': current_date + timedelta(days=random.randint(1, 5))
        }
        usage_data.append(usage)
        
        # Move to next month
        if current_date.month == 12:
            current_date = current_date.replace(year=current_date.year + 1, month=1)
        else:
            current_date = current_date.replace(month=current_date.month + 1)

In [25]:
usage_df = pd.DataFrame(usage_data)

In [26]:
# 7. SALESFORCE-like OPPORTUNITIES
print("Generating sales opportunities data...")
opportunities = []
opp_stages = ['Prospecting', 'Qualification', 'Proposal', 'Negotiation', 'Closed Won', 'Closed Lost']
lead_sources = ['Inbound', 'Outbound', 'Referral', 'Event', 'Partner']

Generating sales opportunities data...


In [27]:
for i in range(200):  # Generate 200 opportunities
    customer_id = random.choice(customers_df['customer_id'].tolist())
    created_date = fake.date_between(START_DATE, END_DATE)
    
    # Determine stage and close date
    stage = random.choices(opp_stages, weights=[0.1, 0.15, 0.2, 0.15, 0.25, 0.15])[0]
    
    if stage in ['Closed Won', 'Closed Lost']:
        close_date = fake.date_between(created_date, END_DATE)
    else:
        close_date = None
    
    # Estimate deal size based on customer size
    customer_size = customers_df[customers_df['customer_id'] == customer_id]['company_size'].iloc[0]
    if customer_size == '1-10':
        deal_value = random.randint(500, 5000)
    elif customer_size == '11-50':
        deal_value = random.randint(2000, 25000)
    elif customer_size == '51-200':
        deal_value = random.randint(10000, 100000)
    elif customer_size == '201-1000':
        deal_value = random.randint(50000, 500000)
    else:
        deal_value = random.randint(100000, 2000000)
    
    opportunity = {
        'opportunity_id': f"opp_{str(uuid.uuid4())[:8]}",
        'customer_id': customer_id,
        'opportunity_name': f"{customers_df[customers_df['customer_id'] == customer_id]['company_name'].iloc[0]} - Tailscale",
        'stage': stage,
        'lead_source': random.choice(lead_sources),
        'deal_value': deal_value,
        'probability': random.randint(10, 90) if stage not in ['Closed Won', 'Closed Lost'] else (100 if stage == 'Closed Won' else 0),
        'created_date': created_date,
        'close_date': close_date,
        'sales_rep': fake.name()
    }
    opportunities.append(opportunity)

In [28]:
opportunities_df = pd.DataFrame(opportunities)

In [29]:
# Display summary statistics
print("\nDataset Summary:")
print("=" * 50)
print(f"Customers: {len(customers_df):,}")
print(f"Subscriptions: {len(subscriptions_df):,}")
print(f"Active Subscriptions: {len(subscriptions_df[subscriptions_df['status'] == 'active']):,}")
print(f"Invoices: {len(invoices_df):,}")
print(f"Payments: {len(payments_df):,}")
print(f"Usage Records: {len(usage_df):,}")
print(f"Opportunities: {len(opportunities_df):,}")


Dataset Summary:
Customers: 1,000
Subscriptions: 1,200
Active Subscriptions: 942
Invoices: 4,520
Payments: 4,290
Usage Records: 7,882
Opportunities: 200


In [30]:
# Calculate some key metrics
total_revenue = payments_df['amount'].sum()
print(f"\nKey Metrics:")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Average Deal Size: ${total_revenue/len(payments_df):,.2f}")


Key Metrics:
Total Revenue: $72,333,698.40
Average Deal Size: $16,861.00


In [31]:
# Save datasets to CSV files
print("\nSaving datasets to CSV files...")
customers_df.to_csv('customers.csv', index=False)
plans_df.to_csv('subscription_plans.csv', index=False)
subscriptions_df.to_csv('subscriptions.csv', index=False)
invoices_df.to_csv('invoices.csv', index=False)
payments_df.to_csv('payments.csv', index=False)
usage_df.to_csv('usage_data.csv', index=False)
opportunities_df.to_csv('opportunities.csv', index=False)


Saving datasets to CSV files...


In [32]:
print("✅ All datasets generated and saved!")
print("\nFiles created:")
print("- customers.csv")
print("- subscription_plans.csv") 
print("- subscriptions.csv")
print("- invoices.csv")
print("- payments.csv")
print("- usage_data.csv")
print("- opportunities.csv")

✅ All datasets generated and saved!

Files created:
- customers.csv
- subscription_plans.csv
- subscriptions.csv
- invoices.csv
- payments.csv
- usage_data.csv
- opportunities.csv


In [33]:
# Display sample data from each table
print("\n" + "="*60)
print("SAMPLE DATA PREVIEW")
print("="*60)


SAMPLE DATA PREVIEW


In [34]:
print("\n📊 CUSTOMERS (first 3 rows):")
print(customers_df.head(3).to_string(index=False))


📊 CUSTOMERS (first 3 rows):
  customer_id                    company_name company_size company_type   industry country created_at acquisition_channel  is_active
cust_acd87ed1 Rodriguez, Figueroa and Sanchez         1-10      Startup     Retail      CU 2024-10-13   Content Marketing       True
cust_3e0ebba3                      Rhodes PLC        11-50      Startup     Retail      BT 2024-08-17             Partner      False
cust_ed1dd98a                   Johnson-Davis         1-10   Enterprise Technology      HT 2024-07-15             Organic       True


In [35]:
print("\n📋 SUBSCRIPTION PLANS:")
print(plans_df.to_string(index=False))


📋 SUBSCRIPTION PLANS:
   plan_id  plan_name  price_monthly  price_annual  max_devices
  personal   Personal              0             0           20
   premium    Premium              6            60          100
      team       Team              6            60          100
  business   Business             18           180          200
enterprise Enterprise             48           480         1000


In [36]:
print("\n💳 SUBSCRIPTIONS (first 3 rows):")
print(subscriptions_df.head(3).to_string(index=False))


💳 SUBSCRIPTIONS (first 3 rows):
subscription_id   customer_id    plan_id billing_cycle  user_count start_date end_date status created_at updated_at
   sub_83cefbca cust_3f380b72   business        annual         915 2024-07-27     None active 2024-07-27 2024-12-28
   sub_bc8e8bfd cust_78e35adb enterprise       monthly         408 2024-08-11     None active 2024-08-11 2024-09-28
   sub_fb1a76b8 cust_0a9a3d5d    premium       monthly           1 2024-09-26     None active 2024-09-26 2024-11-07


In [37]:
print("\n🧾 INVOICES (first 3 rows):")
print(invoices_df.head(3).to_string(index=False))


🧾 INVOICES (first 3 rows):
  invoice_id subscription_id   customer_id invoice_date   due_date  paid_date  amount  discount_amount  final_amount status billing_period_start billing_period_end created_at
inv_10bfa453    sub_83cefbca cust_3f380b72   2024-07-27 2024-08-26 2024-07-29  164700          16470.0      148230.0   paid           2024-07-27         2025-07-26 2024-07-27
inv_18da6ee3    sub_bc8e8bfd cust_78e35adb   2024-08-11 2024-09-10 2024-08-27   19584           3916.8       15667.2   paid           2024-08-11         2024-09-09 2024-08-11
inv_71c67e3f    sub_bc8e8bfd cust_78e35adb   2024-09-10 2024-10-10 2024-10-07   19584           4896.0       14688.0   paid           2024-09-10         2024-10-09 2024-09-10


In [38]:
print("\n💰 PAYMENTS (first 3 rows):")
print(payments_df.head(3).to_string(index=False))


💰 PAYMENTS (first 3 rows):
  payment_id   invoice_id   customer_id   amount currency payment_method payment_date    stripe_charge_id    status created_at
pay_36e1672f inv_10bfa453 cust_3f380b72 148230.0      USD            ach   2024-07-29 ch_fa75f3ec-d6e8-4d succeeded 2024-07-29
pay_25419bce inv_18da6ee3 cust_78e35adb  15667.2      USD           card   2024-08-27 ch_b7040529-c24f-45 succeeded 2024-08-27
pay_9743a11f inv_71c67e3f cust_78e35adb  14688.0      USD           card   2024-10-07 ch_cbd68e13-9f9e-4e succeeded 2024-10-07


In [39]:
print("\n📈 USAGE DATA (first 3 rows):")
print(usage_df.head(3).to_string(index=False))


📈 USAGE DATA (first 3 rows):
      usage_id subscription_id   customer_id      month  devices_connected  data_transfer_gb  active_users  connection_hours created_at
usage_10b7c08f    sub_83cefbca cust_3f380b72 2024-07-01               2901         339524.63           915        1252211.71 2024-07-04
usage_6314faff    sub_83cefbca cust_3f380b72 2024-08-01               2116         776809.19           915        1055305.78 2024-08-04
usage_945b7df7    sub_83cefbca cust_3f380b72 2024-09-01               2100          45468.79           915         818210.17 2024-09-04


In [40]:
print("\n🎯 OPPORTUNITIES (first 3 rows):")
print(opportunities_df.head(3).to_string(index=False))


🎯 OPPORTUNITIES (first 3 rows):
opportunity_id   customer_id                       opportunity_name       stage lead_source  deal_value  probability created_date close_date     sales_rep
  opp_8b9d0896 cust_f886f85d  Bush, Holland and Fischer - Tailscale Negotiation    Outbound       52105           44   2024-09-07       None  Kenneth Rios
  opp_86afa2c9 cust_6495d7d6                    Cox LLC - Tailscale Negotiation     Inbound     1572573           64   2024-11-01       None   Kevin Terry
  opp_78e6b8ae cust_d50dd54e Griffin, Moore and Hensley - Tailscale  Closed Won    Outbound     1362387          100   2024-11-22 2024-12-18 Eric Anderson


In [41]:
print("\n" + "="*60)
print("SUGGESTED DBT MODEL STRUCTURE FOR PRACTICE")
print("="*60)


SUGGESTED DBT MODEL STRUCTURE FOR PRACTICE


In [42]:
dbt_suggestions = """
Suggested dbt models to build with this data:

STAGING MODELS (stg_):
- stg_customers.sql
- stg_subscriptions.sql  
- stg_invoices.sql
- stg_payments.sql
- stg_usage_data.sql
- stg_opportunities.sql

INTERMEDIATE MODELS (int_):
- int_subscription_metrics.sql (MRR, churn calculations)
- int_customer_lifecycle.sql (acquisition, expansion, churn events)
- int_revenue_recognition.sql (deferred revenue calculations)

MART MODELS (fct_ and dim_):
- dim_customers.sql
- dim_subscriptions.sql
- fct_revenue_monthly.sql
- fct_arr_movements.sql
- fct_customer_metrics.sql

BUSINESS LOGIC TO IMPLEMENT:
- Monthly Recurring Revenue (MRR) calculations
- Annual Recurring Revenue (ARR) tracking
- Customer cohort analysis
- Churn rate calculations
- Revenue recognition (deferred revenue)
- Customer Lifetime Value (CLV)
- Net Revenue Retention (NRR)
- Gross Revenue Retention (GRR)

This dataset mimics the complexity you'd find at a SaaS company like Tailscale,
with subscription billing, usage tracking, and sales pipeline data.
"""

In [43]:
print(dbt_suggestions)


Suggested dbt models to build with this data:

STAGING MODELS (stg_):
- stg_customers.sql
- stg_subscriptions.sql  
- stg_invoices.sql
- stg_payments.sql
- stg_usage_data.sql
- stg_opportunities.sql

INTERMEDIATE MODELS (int_):
- int_subscription_metrics.sql (MRR, churn calculations)
- int_customer_lifecycle.sql (acquisition, expansion, churn events)
- int_revenue_recognition.sql (deferred revenue calculations)

MART MODELS (fct_ and dim_):
- dim_customers.sql
- dim_subscriptions.sql
- fct_revenue_monthly.sql
- fct_arr_movements.sql
- fct_customer_metrics.sql

BUSINESS LOGIC TO IMPLEMENT:
- Monthly Recurring Revenue (MRR) calculations
- Annual Recurring Revenue (ARR) tracking
- Customer cohort analysis
- Churn rate calculations
- Revenue recognition (deferred revenue)
- Customer Lifetime Value (CLV)
- Net Revenue Retention (NRR)
- Gross Revenue Retention (GRR)

This dataset mimics the complexity you'd find at a SaaS company like Tailscale,
with subscription billing, usage tracking, an